SAT version for the MCP problem given in the Combinatorial Decision Making and Optimization course.

The model is based on the one already developed for the CP version of the problem with the necessary modifications to make it work for the SAT version.

Necessary libraries:

In [1]:
!pip3 install z3-solver

Necessary imports:

In [2]:
from z3 import * # The Z3 Theorem Prover
import numpy as np # Numpy for matrix operations
import matplotlib.pyplot as plt # Matplotlib for plotting

The variable instances (like number of couriers) are defined in a .dat file. The file is read and the variables are defined.

In [3]:
# open the file in Instances folder
f = open("Instances/inst03.dat", "r")
# the first line is the number of couriers
m = int(f.readline())
# the second line is the number of items
n = int(f.readline())
# the third line is the load size of each courier
load_size = [int(x) for x in f.readline().split()]
# the fourth line is the size of each item
item_size = [int(x) for x in f.readline().split()]
# the rest is the distance matrix
distance = []
for i in range(n+1):
    distance.append([int(x) for x in f.readline().split()])
# close the file
f.close()
print("couriers:", m)
print("items:", n)
print("load_size:", load_size)
print("item_size:", item_size)
# output the distance matrix as a numpy array
distance = np.array(distance)
print("distance:\n", distance)

couriers: 3
items: 7
load_size: [15, 10, 7]
item_size: [3, 2, 6, 8, 5, 4, 4]
distance:
 [[0 3 3 6 5 6 6 2]
 [3 0 6 3 4 7 7 3]
 [3 4 0 7 6 3 5 3]
 [6 3 7 0 5 6 7 4]
 [5 4 6 3 0 3 3 3]
 [6 7 3 6 3 0 2 4]
 [6 7 5 6 3 2 0 4]
 [2 3 3 4 3 4 4 0]]


Other variables:

In [5]:
COURIERS = np.arange(m)
ITEMS = np.arange(n)
items_total = sum(item_size)
# ITEMS_PATH = np.arange(1, max_path_length-1)
# path array 
paths = [[Bool(f'item_{c}_{i}') for i in ITEMS] for c in COURIERS]

In [6]:
paths

[[item_0_0, item_0_1, item_0_2, item_0_3, item_0_4, item_0_5, item_0_6],
 [item_1_0, item_1_1, item_1_2, item_1_3, item_1_4, item_1_5, item_1_6],
 [item_2_0, item_2_1, item_2_2, item_2_3, item_2_4, item_2_5, item_2_6]]

In [8]:
x = Int('x')
y = Int('y')
Distinct(x, y)
x != y
z = Int('z')
Distinct(x, y, z)
Distinct(x, y, z)
simplify(Distinct(x, y, z))
Distinct(x, y, z)
simplify(Distinct(x, y, z), blast_distinct=True)

And(Not(x == y), Not(x == z), Not(y == z))

Formulation of the constraints:

In [7]:
constraints = []
I =  Bool(1)
# the constraint are the same as the CP model version 2
# number of items > number of couriers
constraints.append(n > m)

# Sum of item sizes cannot be bigger than total load size
constraints.append(items_total <= Sum([load_size[i] for i in COURIERS]))


# each item should have size > 0
for i in ITEMS:
    constraints.append(item_size[i] > 0)

# each courier should have load size > 0
for c in COURIERS:
    constraints.append(load_size[c] > 0)

# Each item is delivered at most by one courier, so each column should have at most one 1
for i in ITEMS:
    constraints.append(Distinct(paths[:,i]))

# Each courier delivers at least one item
for c in COURIERS:
    constraints.append(Or([paths[c][p] != n for p in PATH]))
    # constraints.append(Sum([If(paths[c][p] != n, 1, 0) for p in ITEMS_PATH]) >= 1)

# First element and last element in path is the origin (n+1)
# for c in COURIERS:
    #constraints.append(paths[c][0] == n )
    # constraints.append(paths[c][max_path_length - 1] == n )

# Each courier's items sizes must not excede his load size
for i in ITEMS:
    for c in COURIERS:
        constraints.append(Sum([If(paths[c][p] == i, item_size[i], 0) for p in PATH]) <= load_size[c])
    
constraints
# print number of constraints
print(len(constraints))

IndentationError: expected an indented block after 'for' statement on line 20 (186837946.py, line 24)

In [38]:
x = Int('x')
y = Int('y')
print (simplify(x + y + 2*x + 3))
print (simplify(x < y + x + 2))

3 + 3*x + y
Not(y <= -2)


In [35]:
constraints

[True,
 True,
 And(paths_0_1 >= 0, paths_0_1 <= 7),
 And(paths_0_2 >= 0, paths_0_2 <= 7),
 And(paths_0_3 >= 0, paths_0_3 <= 7),
 And(paths_1_1 >= 0, paths_1_1 <= 7),
 And(paths_1_2 >= 0, paths_1_2 <= 7),
 And(paths_1_3 >= 0, paths_1_3 <= 7),
 And(paths_2_1 >= 0, paths_2_1 <= 7),
 And(paths_2_2 >= 0, paths_2_2 <= 7),
 And(paths_2_3 >= 0, paths_2_3 <= 7),
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 If(paths_0_0 == 0, 1, 0) +
 If(paths_0_1 == 0, 1, 0) +
 If(paths_0_2 == 0, 1, 0) +
 If(paths_0_3 == 0, 1, 0) +
 If(paths_0_4 == 0, 1, 0) +
 If(paths_1_0 == 0, 1, 0) +
 If(paths_1_1 == 0, 1, 0) +
 If(paths_1_2 == 0, 1, 0) +
 If(paths_1_3 == 0, 1, 0) +
 If(paths_1_4 == 0, 1, 0) +
 If(paths_2_0 == 0, 1, 0) +
 If(paths_2_1 == 0, 1, 0) +
 If(paths_2_2 == 0, 1, 0) +
 If(paths_2_3 == 0, 1, 0) +
 If(paths_2_4 == 0, 1, 0) ==
 1,
 If(paths_0_0 == 1, 1, 0) +
 If(paths_0_1 == 1, 1, 0) +
 If(paths_0_2 == 1, 1, 0) +
 If(paths_0_3 == 1, 1, 0) +
 If(paths_0_4 == 1, 1, 0) +
 If(paths

Solving the model:

In [36]:
# add all the constraints to the solver
s = Solver()
s.add(constraints)
# solve
if s.check() == sat:
    m = s.model()
    # print the model in a readable format
    paths_values = []
    for i in range(len(paths)):
        paths_values.append([m.evaluate(paths[i][j]) for j in range(len(paths[i]))])
    # convert paths_values to a list of lists of integers
    print(paths_values)
    paths_values = [[int(str(paths_values[i][j])) for j in range(len(paths_values[i]))] for i in range(len(paths_values))]
    # remove from paths_values elements that are equal to n
    paths_values = [[paths_values[i][j] for j in range(len(paths_values[i])) if paths_values[i][j] != n] for i in range(len(paths_values))]
    for c in COURIERS:
        print("Courier", c+1, ":", end=" ")
        print(paths_values[c])

else:
    print("unsat")

[[3, 1, 7, 7, 8], [8, 7, 5, 6, 2], [0, 7, 7, 7, 4]]
Courier 1 : [3, 1, 8]
Courier 2 : [8, 5, 6, 2]
Courier 3 : [0, 4]
